In [1]:
import pandas as pd
import sys
sys.path.append("/root/workSpace/investProject/ConvertBondWheel")

In [2]:
from policy.double_low_enhance.alarm import *

/root/workSpace/investProject/ConvertBondWheel


In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO)

In [8]:
#对应配置 /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/config.py
double_low_conf = {
    'db_topk': 0.1,
    'min_curr_iss_amt': 9,  # <剩余规模
    'volatility': 0.1,  # >波动率
    'premium_rt': 0.2,  # <溢价率
    'min_price': 115,  # <可转债价格
    'min_dblow': 125,  # <双低价格
    'redeem': ['Y'],  #  Y=强赎 不在强赎
    'min_year_left': 1,  # >剩余到期年限 (不要一年内到期的)
    'ytm_rt': -0.01,  # > 到期收益率
    'pluse_out_step':10, # 脉冲轮出时， 轮出债比轮入新债双低值大多少
    'pulse_out_price_median':110, # 脉冲轮出时， 市场价格中位数
    'pulse_out_price':[120, 125], # 脉冲轮出时< 110 和>110 对应的轮出价格
    'pulse_out_dblow':[125, 130],  #同上， 双低值
    'user_zh_columns':[u'转债名称', u'双低', u'现价', u'溢价率', u'剩余规模', u'波动率', u'涨跌幅', u'成交额(万元)', u'换手率'],
} 

#user_zh_columns
# 展示字段， 可以继续添加，
# 如果报错， 说明没有爬取到该字段， 可以通过以下步骤来新增：
# 1. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/fields_config.py
# jisilu_dict 新增对应key-value, jisulu_used_columns 新增对应英文名
# 2. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/items.py
# 添加  该字段 = scrapy.Field()

In [18]:
# 读当前可转债数据
bound = read_data()
balance = read_balancing()

#轮动策略
in_bound = periodWheel(bound,double_low_conf)

# 结果换成中文header & 按列排序
result = display(bound, balance, in_bound, None, user_zh_columns=double_low_conf['user_zh_columns'])
result[:10]

,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,109.63,99.39,10.24%,9.50,28.04%,0.16%,1727.09,1.83%,128127,文科园林,4.40,-0.23%,0.91,4.88,90.16,AA-,3.42,6.34,48.50%,2021-03-01,26-08-19,5.21,3.99%,3.25%,X
1,*靖远转债,110.05,99.91,10.14%,28.00,45.26%,-0.64%,4133.90,1.48%,127027,靖远煤电,2.93,-1.68%,0.79,3.23,90.71,AA+,2.26,4.20,41.80%,2021-06-16,26-12-09,5.52,2.68%,2.17%,X
2,*东湖转债,110.99,104.37,6.62%,15.50,31.28%,0.02%,1729.17,1.07%,110080,东湖高新,6.03,-0.50%,0.89,6.16,97.89,AA,4.31,8.01,33.10%,2021-10-18,27-04-12,5.86,1.71%,1.24%,X
3,洪涛转债,112.16,106.08,6.08%,11.59,29.95%,-0.22%,992.61,0.81%,128013,洪涛股份,2.32,-1.69%,1.04,2.32,100.00,AA-,1.62,3.02,52.50%,2017-02-06,22-07-29,1.15,3.08%,1.46%,X
4,*中装转2,112.42,101.02,11.40%,11.60,39.85%,-0.05%,3433.81,2.93%,127033,中装建设,5.74,-0.52%,1.23,6.33,90.68,AA,4.43,8.23,31.70%,2021-10-22,27-04-15,5.87,2.60%,2.06%,X
5,*孚日转债,113.16,103.41,9.75%,6.50,32.15%,-0.19%,652.33,0.97%,128087,孚日股份,4.24,-0.24%,1.06,4.50,94.22,AA,3.15,5.85,16.90%,2020-06-23,25-12-17,4.54,1.99%,1.46%,X
6,海亮转债,115.84,110.51,5.33%,30.06,29.35%,-0.59%,2024.80,0.61%,128081,海亮股份,10.24,-0.29%,1.98,9.76,104.92,AA,6.83,12.69,15.90%,2020-05-27,25-11-21,4.47,0.89%,0.29%,X
7,*灵康转债,115.88,108.44,7.44%,5.25,38.27%,0.36%,1992.76,3.49%,113610,灵康药业,8.69,0.12%,4.09,8.61,100.93,AA-,6.03,11.19,8.50%,2021-06-07,26-11-30,5.49,2.07%,1.40%,X
8,*鸿达转债,115.97,116.25,-0.28%,8.41,48.04%,2.79%,33328.17,34.39%,128085,鸿达兴业,4.57,3.16%,1.54,3.92,116.58,BB,2.74,5.10,6.30%,2020-06-22,25-12-16,4.54,1.54%,0.62%,X
9,长集转债,116.39,108.47,7.92%,8.00,40.56%,-8.00%,163716.07,182.96%,128105,长青集团,7.95,4.61%,2.10,7.91,100.51,AA,5.54,10.28,21.40%,2020-10-15,26-04-08,4.85,1.22%,0.66%,X


In [14]:
result.describe()

,双低,现价,剩余规模,成交额(万元),代码,正股价,PB,转股价,转股价值,强赎触发价,剩余年限
count,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00
mean,119.39,105.87,5.20,1350.15,120115.64,9.86,2.06,10.58,93.37,13.76,5.03
std,3.50,2.27,2.15,3136.34,6721.20,4.62,0.75,4.94,4.33,6.43,0.83
min,113.44,102.40,2.22,100.93,110071.00,4.15,1.04,4.50,87.78,5.85,3.04
25%,116.48,103.69,3.05,422.47,113593.00,7.96,1.60,8.66,89.13,11.26,4.61
50%,120.13,105.93,5.37,478.73,123092.00,8.66,2.06,9.13,93.33,11.88,5.15
75%,122.58,108.17,7.02,690.65,126053.50,10.04,2.25,11.24,96.03,14.61,5.63
max,124.00,108.84,8.80,12213.09,128087.00,23.53,4.13,24.82,101.86,32.27,5.92
